In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 3 – Merging and Pivoting

## DSC 80, Spring 2024

### Due Date: Wednesday, April 24th at 11:59PM

## Instructions

Welcome to the third DSC 80 lab this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2024-sp).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.** More details on its usage are given at the bottom of this notebook.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from lab import *

In [4]:
import os
import io
from pathlib import Path
import pandas as pd
import numpy as np

<div class="alert alert-block alert-danger" markdown="1">

**There are only two functions in this lab in which you may use a `for`-loop:**
- **`read_linkedin_survey` in Question 1.**
- **`read_student_surveys` in Question 2.**
    
**You may lose points if you use a `for`-loop or `while`-loop in any other question!**

</div>

## Part 1: Combining Data

### Question 1 – Making Connections 🤝

A group of students decided to send out a survey to their connections on LinkedIn. Each student asks 1000 of their connections for their first and last name, the company they currently work at, their job title, their email, and the university they attended.

**Your job is to combine all the data contained in the files `survey*.csv` (stored within the `data/responses` folder) into a single DataFrame. The number of files and the number of rows in each file may vary, so don't hardcode your answers!** To do so, implement the following two functions.

#### `read_linkedin_survey`

Complete the implementation of the function `read_linkedin_survey`, which takes in a Path for the directory where `survey*.csv` files are located and outputs a single DataFrame with six columns titled `'first name'`, `'last name'`, `'current company'`, `'job title'`, `'email'`, and `'university'` (in that order) containing the all of survey information from all the `.csv` files. Make sure to reset the index of the combined DataFrame before returning it so that the index is unique. 

***Hints***:

- Take a look at a few of the files in the `responses` folder. You may have to do some data cleaning to combine the DataFrames!

- You can list the files in a directory using `.iterdir()`.

***Note***: Remember to use `Path()` to build file paths.

<br>

#### `com_stats`

Complete the implementation of the function `com_stats`, which takes in a DataFrame returned by `read_linkedin_survey` and returns a list containing, in the following order: 
- The proportion of people who went to a university with the string `'Ohio'` in its name that have the string `'Programmer'` somewhere in their job title.
- The number of job titles that **end** with the exact string `'Engineer'`. Note that we're asking for the number of job titles, **not** the number of people!
- The job title that has the longest name (there are no ties).
- The number of people who have the word `'manager'` in their job title, uppercase or lowercase (`'Manager'`, `'manager'`, and `'mANAgeR'` should all count).

In [5]:
def read_linkedin_survey(dirname):
    survey_df = []
    cols = ['first name', 'last name', 'current company', 'job title', 'email', 'university']            

    for file_path in Path(dirname).iterdir():
        if file_path.is_file():
            survey = pd.read_csv(file_path)
            survey.columns = [col.lower().replace('_', ' ') for col in survey.columns]
            survey = survey[cols]
            survey_df.append(survey)
            
    combine_df = pd.concat(survey_df, ignore_index=True)
    
    combine_df.reset_index(drop=True, inplace=True)
    
    return combine_df

In [6]:
def com_stats(df):
    df_cleaned = df.dropna(subset=['university', 'job title'])
    ohio_denom = df_cleaned[df_cleaned['university'].str.contains('Ohio', case=False)].shape[0]
    ohio = df_cleaned[df_cleaned['university'].str.contains('Ohio', case=False) & df_cleaned['job title'].str.contains('Programmer', case=False)].shape[0]
    
    unique_job = df_cleaned['job title'].unique()
    last_words = pd.Series(unique_job).str.split().str[-1]
    engineer = (last_words == 'Engineer').sum()

    longest = max(unique_job, key=len)
    
    manager = df_cleaned[df_cleaned['job title'].str.contains('manager', case=False)].shape[0]

    return [ohio/ohio_denom,int(engineer),str(longest),manager]

In [7]:
# stats_out = com_stats(q1_out)
# stats_out

In [8]:
# dirname = Path('data') / 'responses'
# q1_out = read_linkedin_survey(dirname)
# q1_out

In [9]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'responses'
q1_out = read_linkedin_survey(dirname)
stats_out = com_stats(q1_out)

In [10]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2 – Survey Says... 👨‍👩‍👧‍👦

Professor Aritra often sends out extra credit surveys asking students for their favorite animals, movies, and other favorite things. These surveys are stored in the `data/extra-credit-surveys` folder. Each file in that folder corresponds to a different survey question (except for `favorite1.csv`, which contains students' names and IDs).

Here's how extra credit works:
- Each student who has completed at least 50% of the survey questions receives 5 points of extra credit.
- If there is at least one survey question that at least 90% of the class answered (e.g. favorite animal), **everyone** in the class receives 1 point of extra credit. This overall class extra credit only applies twice, so if for example 95% of students answer the favorite color survey question and 91% answer the favorite animal survey question, and and 97% answer the favorite movie question, the entire class still receives 2 extra point as a class, not 3.
- Note that this means that the most extra credit any student can earn is 7 points.

#### `read_student_surveys`

Complete the implementation of the function `read_student_surveys` which takes in a Path describing the directory to a folder containing `favorite*.csv` files and outputs a DataFrame containing all of the survey data combined, indexed by student ID (a value 1-1000).

***Note***: Remember to use `Path()` to build file paths.


<br>

#### `check_credit`

Complete the implementation of the function `check_credit` which takes in a DataFrame returned by `read_student_surveys` and outputs a DataFrame indexed by student ID (a value 1-1000) with two columns:
- `'name'`, containing the name of each student, and
- `'ec'`, containing the number of extra credit points each student earned.

***Note***: For the genres survey, `'(no genres listed)'` does not count as a valid response for receiving extra credit. 

In [11]:
def read_student_surveys(dirname):
    ec_question =[]
    for file_path in Path(dirname).iterdir():
        if file_path.is_file():
            ec = pd.read_csv(file_path)
            ec_question.append(ec)

        ec_df = pd.concat(ec_question, axis=1, join='outer')

        # Drop duplicate 'student_id' columns
        ec_df = ec_df.loc[:, ~ec_df.columns.duplicated()]
        ec_df.drop(columns=['id'], inplace=True)
        
    return ec_df

In [12]:
# q2_out.count(axis=1)/len(q2_out.columns)

# q2_out.count()/len(q2_out)
# len(q2_out.columns)
# no_name = q2_out.drop(columns=['name'])

# free = (no_name.count()/len(no_name) >= 0.9).sum()
# if (free > 2):
#     free = 2
# else:
#     free = np.floor(free)
# free

In [13]:
# dirname = Path('data') / 'extra-credit-surveys'
# q2_out = read_student_surveys(dirname)
# q2_out

# column_to_move = 'name'

# # Get the index of the column to move
# column_index = q2_out.columns.get_loc(column_to_move)

# # Move the column to the first position
# q2_out.insert(0, column_to_move, q2_out.pop(column_to_move))

# q2_out['name']

In [14]:
def check_credit(df):
    df.replace('(no genres listed)', np.nan, inplace=True)

    num_questions = len(df.columns)-1
    no_name = df.drop(columns=['name'])
    free = (no_name.count()/len(no_name) >= 0.9).sum()
    if (free > 2):
        free = 2
    else:
        free = np.floor(free)
        
    participation = (no_name.count(axis=1)/num_questions)
    ec_grade = participation.apply(lambda x: 5 + free if x >= 0.5 else free)
    check_df = pd.DataFrame({'name': df['name'],'ec': ec_grade})

        
    return check_df

In [15]:
# check_credit_out = check_credit(q2_out)
# check_credit_out

In [16]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'extra-credit-surveys'
q2_out = read_student_surveys(dirname)
check_credit_out = check_credit(q2_out)

In [17]:
# check_credit_out['ec'].max()# == 6

In [18]:
grader.check("q2")

q2 results: All test cases passed!

### Question 3 – Paw Patrol 🐾

You are analyzing data from a veterinarian clinic. The datasets contain several types of information from the clinic, including its customers (pet owners), pets, available procedures, and procedure history. The column names are self-explanatory. These DataFrames are provided to you:
-  `owners` stores the customer information, where every `'OwnerID'` is unique (verify this yourself).
-  `pets` stores the pet information. Each pet belongs to a customer in `owners`.
-  `procedure_detail` contains a catalog of procedures that are offered by the clinic.
-  `procedure_history` has procedure records. Most procedures were given to a pet in `pets`.

Complete the implementation of the following three functions, which each ask you to answer a specific question.

#### `most_popular_procedure`

What is the most popular `'ProcedureType'` amongst all pets in the `pets` DataFrame? Complete the implementation of the function `most_popular_procedure`, which takes in two DataFrames, `pets` and `procedure_history`, and returns the name of the most popular `'ProcedureType'` as a string.

Note that some pets are registered but haven't had any procedures performed. Also, some pets that have had procedures done are not registered in `pets`.

<br>

#### `pet_name_by_owner`

What is the name of each customer's pet(s)? Complete the implementation of the function `pet_name_by_owner`, which takes in two DataFrames, `owners` and `pets`, and returns a Series whose index contains owner first names, and whose values are pet names as **strings**. If an owner has multiple pets, the value corresponding to that owner should instead be a **list of pet names as strings**.

Note that owner first names are not necessarily unique, and so the Series you return will not necessarily have a unique index.

<br>

#### `total_cost_per_city`

Note that the `owners` DataFrame has a `'City'` column, describing the city in which each pet owner and their pets live. How much did each city spend in total on procedures? Complete the implementation of the function `total_cost_per_city`, which takes in four DataFrames, `owners`, `pets`, `procedure_history`, and `procedure_detail`, and returns a Series indexed by `'City'` that describes the total amount that each city has spent on pets' procedures.

***Hint***: At some point, you may have to merge on multiple columns.

***Note***: Some owners may have never visited the veterinarian clinic in their city. This means some cities may have zero operational costs.

In [23]:
def most_popular_procedure(pets, procedure_history):
    pop_procedure = pd.merge(pets, procedure_history, on = 'PetID', how = 'left')
    procedure = pop_procedure['ProcedureType'].value_counts()
    most_pop = procedure.idxmax()
    return most_pop

In [48]:
def pet_name_by_owner(owners, pets):
    owners_pets = pd.merge(owners, pets, on='OwnerID', how='left')
    
    second_group = owners_pets.groupby(['Name_x','OwnerID']) 
    
    pet = second_group['Name_y'].agg(lambda x: x.tolist() if len(x) > 1 else x.iloc[0])
    pet = pet.reset_index().set_index('Name_x')['Name_y']

    return pet

In [61]:
def total_cost_per_city(owners, pets, procedure_history, procedure_detail):
    owners_pets = pd.merge(owners, pets, on='OwnerID', how='left')
    
    history = pd.merge(owners_pets, procedure_history, on='PetID', how='left')
    
    merged_df = pd.merge(history, procedure_detail, on='ProcedureType', how='left')
    
    total_cost_per_city = merged_df.groupby('City')['Price'].sum()
    
    return total_cost_per_city

In [62]:
# do not edit this cell -- it is needed for the tests
pets_fp = Path('data') / 'pets' / 'Pets.csv'
procedure_history_fp =  Path('data') / 'pets' / 'ProceduresHistory.csv'
owners_fp = Path('data') / 'pets' / 'Owners.csv'
procedure_detail_fp = Path('data') / 'pets' / 'ProceduresDetails.csv'
pets = pd.read_csv(pets_fp)
procedure_history = pd.read_csv(procedure_history_fp)
owners = pd.read_csv(owners_fp)
procedure_detail = pd.read_csv(procedure_detail_fp)

out_01 = most_popular_procedure(pets, procedure_history)
out_02 = pet_name_by_owner(owners, pets)
out_03 = total_cost_per_city(owners, pets, procedure_history, procedure_detail)

In [63]:
# merged_owners_pets = pd.merge(owners, pets, on='OwnerID', how='left')
    
# # Merge merged_owners_pets with procedure_history DataFrame on 'PetID'
# merged_procedure_history = pd.merge(merged_owners_pets, procedure_history, on='PetID', how='left')
    
# # Merge merged_procedure_history with procedure_detail DataFrame on 'ProcedureType'
# merged_data = pd.merge(merged_procedure_history, procedure_detail, on='ProcedureType', how='left')
    
# # Group by 'City' and aggregate total cost per city
# total_cost_per_city = merged_data.groupby('City')['Price'].sum()
    
# total_cost_per_city

In [64]:
grader.check("q3")

q3 results: All test cases passed!

## Part 2: Pivot Tables

### Question 4 – Summarizing Sales 💰

Recall from [Lecture 3](https://dsc80.com/resources/lectures/lec03/lec03-filled.html#Pivot-tables-using-the-pivot_table-method), a pivot table allows you to aggregate the entries in a DataFrame based on two categorical columns. In this question, you are given a simple dataset, `data/sales.csv`, and are asked to solve a few problems using the `pivot_table` method. 

**We have provided outlines for the DataFrames you need to create, but yours may have a different number of rows and columns and different values.**

***Note***: If it helps, [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html) is another great resource that provides an overview of `pivot_table` with many examples from the Titanic dataset.


#### `average_seller`

Complete the implementation of the function `average_seller`, which takes in the `sales` DataFrame and returns a DataFrame that contains the average sales for each seller, indexed by `'Name'` and containing the column `'Average Sales'`. There should not be any `NaN`s.

***Note***: You may be able to implement `average_seller` without using `pivot_table`.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Average Sales</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `product_name`

Complete the implementation of the function `product_name` that takes in the `sales` DataFrame and returns a DataFrame that contains the total sales for each product, indexed by `'Name'`. Do not fill in `NaN`s.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th>Product</th>
      <th>boat</th>
      <th>book</th>
      <th>hotel</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>NaN</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>NaN</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
  </tbody>
</table>

<br>

#### `count_product`

Complete the implementation of the function `count_product` that takes in the `sales` DataFrame and returns a DataFrame that contains the total number of items sold product-wise and name-wise per date. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Date</th>
      <th>01.01.2012</th>
      <th>02.20.2013</th>
      <th>02.25.2015</th>
    </tr>
    <tr>
      <th>Product</th>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>boat</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">book</th>
      <th>Jones</th>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>hotel</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `total_by_month`

Complete the implementation of the function `total_by_month` that takes in the `sales` DataFrame and returns a pivot table that contains the total sales name-wise, product-wise per month. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Month</th>
      <th>February</th>
      <th>January</th>
      <th>July</th>
      <th>March</th>
    </tr>
    <tr>
      <th>Name</th>
      <th>Product</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="3" valign="top">Jones</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">Smith</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

In [114]:
def average_seller(sales):
    pivot = pd.pivot_table(sales, values='Total', index='Name', aggfunc='mean', fill_value=0)
    sales_avg = pivot.reset_index()
    sales_avg.columns = ['Name', 'Average Sales']
    
    return sales_avg

def product_name(sales):
    return sales.pivot_table(index='Name',
               columns='Product',
               values='Total',
               aggfunc='sum')

def count_product(sales):
    return sales.pivot_table(index=['Product','Name'],
               columns='Date',
               values='Total',
               aggfunc='count', fill_value=0)

def total_by_month(sales):
    sales['Month'] = sales['Date'].str.split('.').str[0].astype(int)
    month = {
        1: 'January', 2: 'February', 3: 'March', 4: 'April',
        5: 'May', 6: 'June', 7: 'July', 8: 'August',
        9: 'September', 10: 'October', 11: 'November', 12: 'December'
    }
    
    sales['Month'] = sales['Month'].map(month)

    total_month = sales.pivot_table(index=['Name', 'Product'], columns='Month', values='Total', aggfunc='sum', fill_value=0)
    return total_month

In [115]:
# don't change this cell -- it is needed for the tests to work
fp = Path('data') / 'sales.csv'
sales = pd.read_csv(fp)
q4_average_seller_out = average_seller(sales)
q4_product_name_out = product_name(sales)
q4_product_count_out = count_product(sales)
q4_total_by_month_out = total_by_month(sales)

In [116]:
q4_total_by_month_out

Month          February  January  July  March   May
Name  Product                                      
Jones book           30        0     0      0     0
      pen           300        0     0      0     0
      ruler        3350        0     0      0     0
Smith book            0      200     0      0  2000
      pen             0     2500     0      0     0
      ruler           0        0  2100      0     0
Trump boat            0        0     0    700     0
      book            0        0     0   1000     0
      hotel           0        0     0    100     0
      pen             0        0     0    150     0

In [117]:
# # sales['Month'] = sales['Date'].dt.month
# (sales['Date'])

In [118]:
sales

,Name,Product,Date,Total,Month
0,Smith,book,01.01.2012,200,January
1,Jones,pen,02.20.2013,300,February
2,Trump,hotel,03.03.2015,100,March
3,Smith,book,05.10.2013,2000,May
4,Jones,book,02.20.2013,30,February
5,Trump,boat,03.30.2017,700,March
6,Smith,ruler,07.05.2014,2100,July
7,Jones,ruler,02.25.2015,350,February
8,Trump,book,03.03.2015,1000,March
9,Smith,pen,01.01.2012,2500,January


In [119]:
grader.check("q4")

q4 results: All test cases passed!

## Congratulations! You're done Lab 3! 🏁

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.

Once you've finished the lab, you should open the command line and run, in the directory for this lab:

```
python lab-validation.py
```

**This will run all of the `grader.check` cells that you see in this notebook, but only using the code in `lab.py` – that is, it doesn't look at any of the code in this notebook. If all of your `grader.check` cells pass in this notebook but not all of them pass in your command line with the above command, then you likely have code in your notebook that isn't in your `lab.py`!**

You can also use `lab-validation.py` to test individual questions. For instance,

```
python lab-validation.py q1 q2 q4
```

will run the `grader.check` cells for Questions 1, 2, and 4 – again, only using the code in `lab.py`. [This video](https://www.loom.com/share/0ea254b85b2745e59322b5e5a8692e91?sid=5acc92e6-0dfe-4555-9b6a-8115b6a52f99) how to use the script as well.

Once `python lab-validation.py` shows that you're passing all test cases, you're ready to submit your `lab.py` (and only your `lab.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

<div class="alert alert-block alert-danger" markdown="1">

**There are only two functions in this lab in which you may use a `for`-loop:**
- **`read_linkedin_survey` in Question 1.**
- **`read_student_surveys` in Question 2.**
    
**You may lose points if you use a `for`-loop or `while`-loop in any other question!**

</div>

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [120]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!